In [ ]:
subject_dir = "insert_subject_dir_here"
outdir = "insert_outdir_here"

In [ ]:
%matplotlib notebook

In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to show/hide code."></form>''')

# HCP-ASL Pre-processing Pipeline Report

This notebook seeks to provide an interactive look at some of the results obtained by running the HCP-ASL Minimal Pre-processing Pipeline. Some useful plots and summary statistics are shown below, with hyperlinks to the different sections so you can jump to the part you're interested in.

This notebook also contains code to produce the cells which is hidden by default. This can be shown by clicking the toggle below which reads "Click here to show/hide code".

This is only meant to provide an initial look at some of the processed data. If you would like to interrogate the results of the pipeline further, it is advised to use tools such as wb_view/FSLEyes/the viewer of your choice.

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import nilearn.plotting
import ipywidgets
from IPython.display import display
import numpy as np
import regtricks as rt
import nibabel as nb
import pandas as pd

subject_name = Path(subject_dir).stem
print(f"Subject name: {subject_name}")

## Contents

[1. Summary](#Summary)

[2. Voxelwise results](#VoxelwiseResults)
   
[3. Surface-projected Results](#SurfaceResults)

[4. Registration Quality](#Registration)

[5. Motion Estimates](#MotionEstimates)

[6. SE-based Bias Correction](#BiasCorrection)

<a id="Summary"></a>

## 1. Summary

Summary stats to show?

* Volumetric Results
    * PVC/Non-PVC
        * Mean GM/WM perfusion
        * Mean GM/WM arrival
        * Spatial Coefficient of Variation in GM/WM perfusion
        * Spatial Coefficient of Variation in GM/WM aCBV
* Surface Results
    * Mean and Spatial Coefficient of Variation of surface-projected perfusion
    * Mean and Spatial Coefficient of Variation of surface-projected arrival

### Volumetric Summary Statistics

In [ ]:
# define summary statistics functions
def masked_summary_stats(image, mask):
    nanned_arr = np.where(mask.get_fdata()==1., image.get_fdata(), np.nan)
    nan_mean = np.nanmean(nanned_arr)
    nan_std = np.nanstd(nanned_arr)
    spatial_cov = nan_std*100 / nan_mean
    return nan_mean, nan_std, spatial_cov

In [ ]:
# voxel-space results directories
oxford_asl_dir = Path(subject_dir)/outdir/"T1w/ASL/OxfordASL/native_space"
pvcorr_dir = oxford_asl_dir/"pvcorr"

# load tissue masks
tissues = ("wm", "gm")
masks = [nb.load(oxford_asl_dir/f"{tissue}_roi.nii.gz") for tissue in tissues]

# load non-pvc estimates
estimate_names = ("arrival", "perfusion_calib", "aCBV_calib")
arrival, pcalib, aCBVcalib = [nb.load(oxford_asl_dir/f"{name}.nii.gz") for name in estimate_names]
(wm_aCBVcalib_stats, gm_aCBVcalib_stats) = [masked_summary_stats(aCBVcalib, mask) for mask in masks]

# calculate summary stats in non-pvc estimates
arrival_wm, arrival_gm = [masked_summary_stats(arrival, mask) for mask in masks]
pcalib_wm, pcalib_gm = [masked_summary_stats(pcalib, mask) for mask in masks]

In [ ]:
# load pvc estimates
wm_pvc_estimate_names = ("arrival_wm_masked", "perfusion_wm_calib_masked")
gm_pvc_estimate_names = ("arrival_masked", "perfusion_calib_masked")
pvc_aCBV_name = "aCBV_calib"

pvc_wm_arrival, pvc_wm_pcalib = [nb.load(pvcorr_dir/f"{name}.nii.gz") for name in wm_pvc_estimate_names]
pvc_gm_arrival, pvc_gm_pcalib = [nb.load(pvcorr_dir/f"{name}.nii.gz") for name in gm_pvc_estimate_names]
pvc_aCBVcalib = nb.load(pvcorr_dir/f"{pvc_aCBV_name}.nii.gz")

# calculate summary stats in pvc estimates
(pvc_arrival_wm_stats, pvc_pcalib_wm_stats), (pvc_arrival_gm_stats, pvc_pcalib_gm_stats) = [
    [masked_summary_stats(image, mask) for image in images]
    for images, mask in zip(((pvc_wm_arrival, pvc_wm_pcalib), (pvc_gm_arrival, pvc_gm_pcalib)), masks)
]
(pvc_wm_aCBVcalib_stats, pvc_gm_aCBVcalib_stats) = [masked_summary_stats(pvc_aCBVcalib, mask) for mask in masks]

In [ ]:
def plot_table(pvc):
    stats = summary_stats[pvc]
    tissues = ("wm", "gm")
    df = pd.DataFrame(stats, columns=list(stats.keys()), index=tissues)
    print(df.head())

# display results
pvc = (("True", "pvc"), ("False", "nonpvc"))
pvc_value = ipywidgets.Dropdown(options=pvc,
                                value="pvc",
                                description="PVC:")

# get summary stats based on value of pvc_value
summary_stats = {
    "pvc": ((pvc_arrival_wm_stats, pvc_pcalib_wm_stats),
            (pvc_arrival_gm_stats, pvc_pcalib_gm_stats),
            (pvc_gm_aCBVcalib_stats)),
    "nonpvc": ((arrival_wm, pcalib_wm),
               (arrival_gm, pcalib_gm),
               (gm_aCBVcalib_stats))
}
summary_stats = {
    "pvc": {"Mean Arrival": (pvc_arrival_wm_stats[0], pvc_arrival_gm_stats[0]),
            "SpCoV Arrival": (pvc_arrival_wm_stats[2], pvc_arrival_gm_stats[2]),
            "Mean Perfusion": (pvc_pcalib_wm_stats[0], pvc_pcalib_gm_stats[0]),
            "SpCoV Perfusion": (pvc_pcalib_wm_stats[2], pvc_pcalib_gm_stats[2]),
            "SpCoV aCBV": (pvc_wm_aCBVcalib_stats[2], pvc_gm_aCBVcalib_stats[2])},
    "nonpvc": {"Mean Arrival": (arrival_wm[0], arrival_gm[0]),
               "SpCoV Arrival": (arrival_wm[2], arrival_gm[2]),
               "Mean Perfusion": (pcalib_wm[0], pcalib_gm[0]),
               "SpCoV Perfusion": (pcalib_wm[2], pcalib_gm[2]),
               "SpCoV aCBV": (wm_aCBVcalib_stats[2], gm_aCBVcalib_stats[2])}
}
show_stats = summary_stats[pvc_value.value]

# display in table
t = ipywidgets.interactive(plot_table,
                           pvc=pvc_value,
                           tissues=ipywidgets.fixed(tissues),
                           stats=ipywidgets.fixed(summary_stats[pvc_value.value]));
display(t);

### Surface-based Summary Statistics

In [ ]:
def surface_summary_stats(func):
    mean = np.mean(func)
    std = np.std(func)
    SpCoV = std*100 / mean
    return mean, std, SpCoV

# find projected results
cifti_dir = Path(subject_dir)/outdir/"T1w/ASL/CIFTIPrepare"
sides = ("L", "R")

# load projected perfusion
perfusion_calibs = [cifti_dir/f"perfusion_calib_s2.atlasroi.{side}.32k_fs_LR.func.gii" for side in sides]
l_perf, r_perf = [nilearn.surface.load_surf_data(str(name)) for name in perfusion_calibs]
surf_perf_stats = [surface_summary_stats(func) for func in (l_perf, r_perf)]

# load projected arrival
arrivals = [cifti_dir/f"arrival_s2.atlasroi.{side}.32k_fs_LR.func.gii" for side in sides]
l_arr, r_arr = [nilearn.surface.load_surf_data(str(name)) for name in arrivals]
surf_arr_stats = [surface_summary_stats(func) for func in (l_arr, r_arr)]

# find PVC projected results
pvc_cifti_dir = Path(subject_dir)/outdir/"T1w/ASL/CIFTIPrepare/pvcorr"

# load projected perfusion
pvc_perfusion_calibs = [pvc_cifti_dir/f"perfusion_calib_s2.atlasroi.{side}.32k_fs_LR.func.gii" for side in sides]
pvc_l_perf, pvc_r_perf = [nilearn.surface.load_surf_data(str(name)) for name in pvc_perfusion_calibs]
pvc_surf_perf_stats = [surface_summary_stats(func) for func in (pvc_l_perf, pvc_r_perf)]

# load projected arrival
pvc_arrivals = [pvc_cifti_dir/f"arrival_s2.atlasroi.{side}.32k_fs_LR.func.gii" for side in sides]
pvc_l_arr, pvc_r_arr = [nilearn.surface.load_surf_data(str(name)) for name in pvc_arrivals]
pvc_surf_arr_stats = [surface_summary_stats(func) for func in (pvc_l_arr, pvc_r_arr)]

In [ ]:
def plot_surface_table(pvc):
    stats = surface_summary_stats[pvc]
    df = pd.DataFrame(stats, columns=list(stats.keys()), index=sides)
    print(df.head())
    
# create table for projected results
surface_summary_stats = {
    "pvc":{"Mean Arrival": [surf_arr[0] for surf_arr in pvc_surf_arr_stats],
            "SpCoV Arrival": [surf_arr[2] for surf_arr in pvc_surf_arr_stats],
            "Mean Perfusion": [surf_perf[0] for surf_perf in pvc_surf_perf_stats],
            "SpCoV Perfusion": [surf_perf[2] for surf_perf in pvc_surf_perf_stats]},
    "nonpvc":{"Mean Arrival": [surf_arr[0] for surf_arr in surf_arr_stats],
              "SpCoV Arrival": [surf_arr[2] for surf_arr in surf_arr_stats],
              "Mean Perfusion": [surf_perf[0] for surf_perf in surf_perf_stats],
              "SpCoV Perfusion": [surf_perf[2] for surf_perf in surf_perf_stats]}
}
show_stats = surface_summary_stats[pvc_value.value]

# display in table
t = ipywidgets.interactive(plot_surface_table,
                           pvc=pvc_value,
                           tissues=ipywidgets.fixed(tissues),
                           stats=ipywidgets.fixed(summary_stats[pvc_value.value]));
display(t);

<a id="VoxelwiseResults"></a>

## 2. Voxelwise Results
Voxelwise estimates for arrival time, perfusion and aCBV (both calibrated) are shown below. You can toggle between showing partial volume corrected and non-partial volume corrected results.

In [ ]:
# set up dictionaries to index when choices are selected in dropdown menus
image_names = ("perfusion_calib.nii.gz", "aCBV_calib.nii.gz", "arrival.nii.gz")
images = {"pvc" : [(name.strip(".nii.gz"), str(pvcorr_dir/name)) for name in image_names],
          "nonpvc" : [(name.strip(".nii.gz"), str(oxford_asl_dir/name)) for name in image_names]}
maxs = {"perfusion_calib.nii.gz": 150.,
        "aCBV_calib.nii.gz": 5.,
        "arrival.nii.gz": 5.}
vals = {"perfusion_calib.nii.gz": 100.,
        "aCBV_calib.nii.gz": 2.5,
        "arrival.nii.gz": 2.5}
cmaps = {"perfusion_calib.nii.gz" : "cold_hot",
         "aCBV_calib.nii.gz" : "cold_hot",
         "arrival.nii.gz" : "gray",
         "calib0_gdc_dc.nii.gz": "gray",
         "sebased_bias_dil.nii.gz": "gray",
         "calib0_secorr.nii.gz": "gray"}

def display_plot(image_name, threshold, fig, x=0, y=0, z=0):
    image_type = image_name.split("/")[-1]
    cmap = cmaps[image_type]
    if cmap == "cold_hot":
        vmin = -threshold
    else:
        vmin = 0.
    title = f"{pvc_value.value} {image_type}"
    fig.clear()
    nilearn.plotting.plot_epi(image_name, cut_coords=(x, y, z), cmap=cmap, figure=fig, 
                              vmin=vmin, vmax=threshold, draw_cross=True, title=title)

def update_image_options(change):
    image_options.options = images[change['new']]

def update_threshold(change):
    image_name = change["new"].split("/")[-1]
    threshold_slider = w.children[1]
    threshold_slider.max = maxs[image_name]
    threshold_slider.value = vals[image_name]
    
# display options for PVC
pvc = (("True", "pvc"), ("False", "nonpvc"))
pvc_value = ipywidgets.Dropdown(options=pvc,
                                value="pvc",
                                description="PVC:")
display(pvc_value);

# display image names for choice of PVC value
image_choices = images[pvc_value.value]
image_options = ipywidgets.Dropdown(options=image_choices,
                                    description="Est Param:")

# update image options if PVC choice changes
pvc_value.observe(update_image_options, names="value")

# plot given current options
fig = plt.figure(figsize=(9, 4))
x, y, z = [ipywidgets.widgets.IntSlider(min=-100, max=100, step=1, value=0, description=l) for l in ("x", "y", "z")]
threshold = ipywidgets.widgets.FloatSlider(min=0.1,
                                           max=maxs[image_options.value.split("/")[-1]],
                                           value=vals[image_options.value.split("/")[-1]],
                                           step=0.1, description="Threshold")
w = ipywidgets.interactive(display_plot,
                           image_name=image_options,
                           threshold=threshold,
                           fig=ipywidgets.fixed(fig),
                           x=x, y=y, z=z);
display(w);

# watch for change in image choice
w.children[0].observe(update_threshold, names="value")

<a id="SurfaceResults"></a>

## 3. Surface-projected Results

### Surface Perfusion

In [ ]:
# find native surfaces
surface_dir = Path(subject_dir)/"T1w/fsaverage_LR32k"
mid_surfaces = [surface_dir/f"{subject_name}.{side}.midthickness_MSMAll.32k_fs_LR.surf.gii" for side in sides]

# load surface files using nilearn and merge left and right
l_mesh, r_mesh = [nilearn.surface.load_surf_mesh(str(surf)) for surf in mid_surfaces]
r_links = r_mesh[1] + len(l_mesh[0])
links = np.concatenate((l_mesh[1], r_links), axis=0)
locs = np.concatenate((l_mesh[0], r_mesh[0] + (10, 0, 0)), axis=0)
meshes = (locs, links)

def plot_surface(surface_mesh, surface_estimate, **kwargs):
    return nilearn.plotting.view_surf(surface_mesh, surface_estimate, **kwargs);


# merge L+R projected perfusion results
perf = np.concatenate((l_perf, r_perf), axis=0)
surf_mean_perf = np.mean(perf)
surf_std_perf = np.std(perf)
surf_perf_vmax = surf_mean_perf + 3*surf_std_perf

plot_surface(meshes, perf, vmax=surf_perf_vmax)

### Surface Arrival Time

In [ ]:
# get projected arrival results and combine into one numpy array for plotting
arr = np.concatenate((l_arr, r_arr), axis=0)
surf_mean_arr = np.mean(arr)
surf_std_arr = np.std(arr)
surf_arr_vmax = surf_mean_arr + 3*surf_std_arr
plot_surface(meshes, arr, cmap="gray", vmin=0., vmax=surf_arr_vmax, symmetric_cmap=False)

<a id="Registration"></a>

## 4. Registration Quality

In [ ]:
def display_registration(image, mask, threshold, fig, x=0, y=0, z=0, **kwargs):
    fig.clear()
    display = nilearn.plotting.plot_epi(image, cut_coords=(x, y, z), figure=fig,
                                        vmin=-threshold, vmax=threshold,
                                        draw_cross=True, **kwargs)
    display.add_contours(mask, levels=[.99], colors=["red"])
    
# load final asl2struct registration
reg_dir = Path(subject_dir)/outdir/"T1w/ASL/TIs/reg"
asl2struct_name = reg_dir/"asl2struct.mat"
perfusion_name = Path(subject_dir)/outdir/"ASL/OxfordASL/native_space/perfusion.nii.gz"
t1w_name = Path(subject_dir)/"T1w/T1w_acpc_dc_restore.nii.gz"
asl2struct_reg = rt.Registration.from_flirt(asl2struct_name,
                                            src=perfusion_name,
                                            ref=t1w_name)
perfusion_t1w = asl2struct_reg.apply_to_image(src=perfusion_name,
                                              ref=t1w_name,
                                              order=1)

# load and print final cost of bbr registration
cost = np.loadtxt(reg_dir/"asl2orig_mgz_initial_bbr.dat.mincost")
print(f"Final cost of asl2orig registration via bbregister: {cost[0]:.5f}")

# get white matter surface outline
wm_mask = Path(subject_dir)/outdir/"T1w/ASL/reg/wmmask.nii.gz"
fig = plt.figure(figsize=(9, 4))
title = "Registered initial perfusion estimate"
cmap = "gray"
threshold = ipywidgets.widgets.FloatSlider(min=0.1,
                                           max=30.,
                                           value=15.,
                                           step=0.1, description="Threshold")
x, y, z = [ipywidgets.widgets.IntSlider(min=-100, max=100, step=1, value=0, description=l) for l in ("x", "y", "z")]
ipywidgets.interact(display_registration,
                    image=ipywidgets.fixed(perfusion_t1w),
                    mask=ipywidgets.fixed(str(wm_mask)),
                    threshold=threshold,
                    fig=ipywidgets.fixed(fig),
                    x=x, y=y, z=z,
                    title=ipywidgets.fixed(title),
                    cmap=ipywidgets.fixed(cmap));

<a id="MotionEstimates"></a>

## 5. Motion Estimates
Plot magnitudes of motion estimates to see if there is a suspicious motion spike.

In [ ]:
def display_motion(par_file, motion_type, motion_ax):
    motion_ax.clear()
    names = motion_dict[motion_type]["DF labels"]
    labels = motion_dict[motion_type]["Labels"]
    xlabel, ylabel = motion_dict[motion_type]["Axis labels"]
    for name, label in zip(names, labels):
        motion_ax.plot(np.arange(0, len(par_file)), par_file[name], label=label)
    motion_ax.legend(loc='upper right')
    motion_ax.set_xlabel(xlabel)
    motion_ax.set_ylabel(ylabel)
    motion_ax.set_title(f"Motion estimates for subject {subject_name}")

def update_motion_plot(change):
    display_motion(par_file, motion_type.value, motion_ax)
    
# load mcflirt's .par file as pandas DataFrame
par_file = pd.read_csv(Path(subject_dir)/outdir/"ASL/TIs/MoCo/final_registration_TIs.nii.gz.par",
                       names=["xrot", "yrot", "zrot", "x", "y", "z"],
                       delimiter="  ")

# set up widget which allows user to choose type of motion to display
motion_types = ("Rotation", "Translation")
motion_type = ipywidgets.widgets.Dropdown(options=motion_types,
                                          value="Rotation",
                                          description="Motion type")
motion_dict = {"Rotation": {"DF labels":("xrot", "yrot", "zrot"),
                            "Labels": (r"$\theta _x$", r"$\theta _y$", r"$\theta _z$"),
                            "Axis labels": ("Slice number", "Rotation (rads)")},
               "Translation": {"DF labels":("x", "y", "z"),
                               "Labels": ("x", "y", "z"),
                               "Axis labels": ("Slice number", "Translation (mm)")}}

# create figure
motion_fig, motion_ax = plt.subplots(1, 1, figsize=(9, 4))

# plot
display_motion(par_file, motion_type.value, motion_ax)
motion_plot = ipywidgets.interactive(display_motion,
                                     par_file=ipywidgets.fixed(par_file),
                                     motion_type=motion_type,
                                     motion_ax=ipywidgets.fixed(motion_ax))
display(motion_plot);
motion_plot.children[0].observe(update_motion_plot, names="value")

<a id="BiasCorrection"></a>

## 6. SE-based Bias Correction
Allow user to toggle between estimated bias field, uncorrected calibration image and corrected calibration image.

In [ ]:
def display_plot(image_name, threshold, fig, x=0, y=0, z=0):
    image_type = image_name.split("/")[-1]
    cmap = cmaps[image_type]
    title = f"{image_type}"
    fig.clear()
    nilearn.plotting.plot_epi(image_name, cut_coords=(x, y, z), cmap=cmap, figure=fig, 
                              vmin=0., vmax=threshold, draw_cross=True, title=title)

def update_threshold(change):
    image_name = change["new"].split("/")[-1]
    threshold_slider = b.children[1]
    threshold_slider.max = bias_maxs[image_name]
    threshold_slider.value = bias_def[image_name]
    
# load calibration image, estimated bias field and corrected calibration image
calib_dir = Path(subject_dir)/outdir/"T1w/ASL/Calib/Calib0"
calib_dcorr = calib_dir/"DistCorr/calib0_gdc_dc.nii.gz"
sebased_bias = calib_dir/"SEbased/sebased_bias_dil.nii.gz"
calib_biascorr = calib_dir/"SEbased/calib0_secorr.nii.gz"
bias_pairs = [(name.name, str(name)) for name in (calib_dcorr, sebased_bias, calib_biascorr)]

# create widget to choose between images
bias_choice = ipywidgets.Dropdown(options=bias_pairs, description="Image:")

# create figure
fig = plt.figure(figsize=(9, 4))

# set up sensible threshold sliders
bias_maxs = {"calib0_gdc_dc.nii.gz": 2000,
             "sebased_bias_dil.nii.gz": 3,
             "calib0_secorr.nii.gz": 2000}
bias_def = {"calib0_gdc_dc.nii.gz": 1300,
            "sebased_bias_dil.nii.gz": 1.5,
            "calib0_secorr.nii.gz": 1300}
threshold = ipywidgets.FloatSlider(min=0.1,
                                   max=bias_maxs[bias_choice.value.split("/")[-1]],
                                   value=bias_def[bias_choice.value.split("/")[-1]],
                                   step=0.1, description="Threshold")
x, y, z = [ipywidgets.widgets.IntSlider(min=-100, max=100, step=1, value=0, description=l) for l in ("x", "y", "z")]
b = ipywidgets.interactive(display_plot,
                           image_name=bias_choice,
                           threshold=threshold,
                           fig=ipywidgets.fixed(fig),
                           x=x, y=y, z=z)
display(b);
b.children[0].observe(update_threshold, names="value")